In [1]:
import pandas as pd

# Carga

In [2]:
ratings = pd.read_csv("Datos/ratings.csv")
ratings.timestamp = pd.to_datetime(ratings.timestamp, unit='s')

In [3]:
movies = pd.read_csv("Datos/movies.csv")

# Valores perdidos

Vemos que en ratings no hay valores perdidos en las columnas `userId`, `movieId` y `timestamp`. Sin embargo, en la columna `rating` sí que los hay (aproximadamente un $2\%$). Dado que el porcentaje es muy bajo, decidimos eliminar esas filas ya que no supone una pérdida significativa de información.

In [4]:
ratings.isna().any()

userId       False
movieId      False
rating        True
timestamp    False
dtype: bool

In [5]:
ratings.rating.isna().sum()

495115

In [6]:
ratings.rating.isna().sum()/len(ratings)*100

1.9804524742805978

In [7]:
# E
ratings.dropna(axis="index", subset=["rating"], inplace=True)

In [8]:
ratings.rating.isna().sum()/len(ratings)*100

0.0

En el caso de movies podemos ver que en la columna `genres` tiene un valor `(no genres listed)` el cual vamos a mantener como un género en sí mismo.

In [9]:
movies.genres.value_counts()

Drama                                       9056
Comedy                                      5674
(no genres listed)                          5062
Documentary                                 4731
Comedy|Drama                                2386
                                            ... 
Animation|Mystery|Thriller                     1
Drama|Film-Noir|Musical|Thriller               1
Adventure|Crime|Mystery|Romance|Thriller       1
Animation|Children|Comedy|Horror               1
Action|Crime|Thriller|Western                  1
Name: genres, Length: 1662, dtype: int64

# Normalización

Dado que nuestro objetivo es realizar una consulta que realiza un promedio, no vemos necesario normalizar los datos.

# Limpieza de outliers

El único caso que vamos a considerar con outliers es la columna `rating` en ratings.

Esta columna solo debería presentar datos en el intervalo $[0.5,5]$ en incrementos de $0.5$. Los valores fuera de este intervalo serán descartados y los que estén dentro pero no sean múlitplo entero de $0.5$ los redondearemos al valor más cercano.

In [10]:
ratings.rating.value_counts()

 4.0     6505008
 3.0     4798205
 5.0     3540037
 3.5     3112562
 4.5     2156153
 2.0     1607802
 2.5     1237333
 1.0      760986
 1.5      391496
 0.5      385181
 1.1        1039
 2.2         998
 3.3         970
 4.4         965
 0.7         953
-3.0         436
 7.0         335
 10.0        324
 8.0         318
 7.5         315
 6.0         308
 5.5         300
 9.5         296
 9.0         296
 6.5         291
 8.5         279
-5.0         247
-1.0         233
-1.5         231
-0.5         228
-4.0         228
-4.5         216
-2.5         210
-2.0         201
Name: rating, dtype: int64

In [11]:
ratings = ratings[(0.5 <= ratings.rating) & (ratings.rating <= 5)]

In [12]:
ratings.rating.value_counts()

4.0    6505008
3.0    4798205
5.0    3540037
3.5    3112562
4.5    2156153
2.0    1607802
2.5    1237333
1.0     760986
1.5     391496
0.5     385181
1.1       1039
2.2        998
3.3        970
4.4        965
0.7        953
Name: rating, dtype: int64

In [13]:
ratings.rating = (ratings.rating*2).round()/2

In [14]:
ratings.rating.value_counts()

4.0    6505008
3.0    4798205
5.0    3540037
3.5    3113532
4.5    2157118
2.0    1608800
2.5    1237333
1.0     762025
1.5     391496
0.5     386134
Name: rating, dtype: int64

# Preparación y unificación de datos

Para facilitar la consulta por género vamos a separar todos los géneros pertenecientes a una película en registros separados. Posteriormente, vamos a unir ambas tablas para poder realizar la consulta, para la cual necesitamos los géneros y las valoraciones.

In [15]:
movies = movies.assign(genres=movies.genres.str.split('|')).explode('genres')

In [16]:
ratings = ratings.merge(movies, on="movieId", how="left")

Eliminamos el resto de columnas que no se utilizarían en la consulta para acelerar la carga de datos y, por tanto, acelerar el tiempo total de consulta.

In [17]:
ratings = ratings[["rating", "timestamp", "genres"]]

In [18]:
ratings.to_csv("Datos/pelis_procesadas.csv")